In [4]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as sk
from glove_routines import *
from text_classifier import *
%load_ext autoreload
%autoreload 2

In [14]:
def cost_function(i,fn, logn, x, y):
    term1 = np.dot(x,y.T)
    term2 = pow(logn-term1,2)
    if(i<25):
        print("T1",np.shape(term1))
        print("T2",np.shape(term2))
    return sum(term2)*1/2
    #return sum(fn*pow(logn - np.dot(x,y),2)) * 1/2

def glove_SGD(embedding_dim = 20, eta = 0.001, alpha = 3 / 4, epochs = 10, nmax = 100, flags=""):
    print("loading cooccurrence matrix")
    with open('data/cooc.pkl', 'rb') as f:
        cooc = pickle.load(f)
    print("{} nonzero entries".format(cooc.nnz))

    print("using nmax =", nmax, ", cooc.max() =", cooc.max())
    print("Parameters : embedding_dim =",embedding_dim,", eta =",eta,", alpha =",alpha,", epochs =",epochs,".")
    print("initializing embeddings")
    xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
    ys = np.random.normal(size=(cooc.shape[1], embedding_dim))
    losses = []
    i=0
    for epoch in range(epochs):
        print("epoch {}".format(epoch))
        for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
            logn = np.log(n)
            fn = min(1.0, (n / nmax) ** alpha)
            x, y = xs[ix, :], ys[jy, :]
            scale = 2 * eta * fn * (logn - np.dot(x, y))
            xs[ix, :] += scale * y
            ys[jy, :] += scale * x
            i+=1
            if(i%10==0):
                if(i<100):
                    print("i",i)
                losses.append(cost_function(i,fn,logn,xs,ys))            
    np.save(str('data/embeddings'+flags), xs)
    return losses

In [15]:
losses = glove_SGD(flags="test",epochs=2)

loading cooccurrence matrix
6496907 nonzero entries
using nmax = 100 , cooc.max() = 207302
Parameters : embedding_dim = 20 , eta = 0.001 , alpha = 0.75 , epochs = 2 .
initializing embeddings
epoch 0
i 10


KeyboardInterrupt: 

In [8]:
print(np.shape(losses))

(12993814,)
